In [1]:
import random

import numpy as np
import random
import keras
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from keras import models
from keras import layers

%matplotlib inline

Using TensorFlow backend.


In [2]:
# number of characters in a word.
# for instance abccba has nb_chars = 6
nb_chars = 4

# number of possible characters used during the encoding.
# for instance abcde leads to 01234 has nb_letters = 5
#nb_letters = 10
nb_letters = 26

# number of words samples to be generated for the training
nb_words = 3000

In [3]:
# total number of combinations
nb_letters**nb_chars

456976

In [4]:
def create_inputs(nb_words, nb_chars, nb_letters):
    '''Create a numpy array of nb_words rows with nb_chars columns each element
    being a random letter of nb_letters (a, b...)'''
    words = np.zeros((nb_words, nb_chars), dtype=int)
    for w in range(nb_words):
        for c in range(nb_chars):
            i = random.randint(0, nb_letters-1)
            words[w, c] = ord('a') + i
    return words


def encrypt(words, nb_words, nb_chars):
    '''Encrypt each element of a numpy array of nb_words rows with nb_chars 
    columns each item with a secret algorithm'''
    
    encrypted_words = words.copy()
    encrypted_words_probs = np.zeros((nb_words, nb_chars, nb_chars))
    
    #val_max = -1
    
    for w in range(nb_words):
        for c in range(nb_chars): # 0,1,2,3,4
            encrypted_words[w,c] = int(words[w,c]) - 49
            val = encrypted_words[w,c] - 48
            
            #if val > val_max:
            #    val_max = val
            
            # add entropy (i.e. mistakes in the encryption)
            #epsilon = random.randint(0, 100)
            #if epsilon == 5 and val != val_max:
            #val +=1
            
            #print('w:',w,', c:',c,', [wc]:', val)
            #encrypted_words_probs[w, c, val ] = 1.0
            encrypted_words[w,c] = val
    return encrypted_words

def print_output(words, nb_words, nb_chars):
    for w in range(nb_words):
        word = ''
        for c in range(nb_chars):
            word += chr(words[w,c])
        print(word)


In [5]:
nb_features = nb_chars

# This returns a tensor
inputs = layers.Input(shape=(nb_chars,), dtype='float32', name='main_input')

# a layer instance is callable on a tensor, and returns a tensor
x = layers.Dense(2048, activation='relu', name='hl_1')(inputs)
#x = layers.Dense(64, activation='relu', name='hl_2')(x)

outputs = []
losses = {}
for o in range(nb_chars):
    name_i = 'output_'+str(o)
    output_i = layers.Dense(nb_letters, activation='softmax', dtype='float32', name=name_i)(x)
    outputs.append(output_i)
    losses[name_i] = 'categorical_crossentropy'

model = keras.models.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='rmsprop',
              loss=losses,
              metrics=['accuracy'])       

In [6]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 4)            0                                            
__________________________________________________________________________________________________
hl_1 (Dense)                    (None, 2048)         10240       main_input[0][0]                 
__________________________________________________________________________________________________
output_0 (Dense)                (None, 26)           53274       hl_1[0][0]                       
__________________________________________________________________________________________________
output_1 (Dense)                (None, 26)           53274       hl_1[0][0]                       
__________________________________________________________________________________________________
output_2 (

In [7]:
def display_x(x):
    words = []
    for w in x:
        word = ''
        for c in w:
            word += chr(c)
        words.append(word)
   
    print(words)

In [8]:
# ALL IN ONE STEP ('OSSIFIED')

x = create_inputs(nb_words, nb_chars, nb_letters)
display_x(x[:3])
print('x:\n', x[:3], 'out of ',len(x))
print()

# process the x data as useful ANN input data
scaler = StandardScaler()
x_train  = scaler.fit_transform(x)

print('x_train:\n', x_train[:3], 'out of ',len(x_train))
print()

# create output data for training
y = encrypt(x, nb_words, nb_chars)
print('y:\n', y)
print()
# process the y data as useful ANN output data
y_train = keras.utils.to_categorical(y, nb_letters)
print('y_train:\n', y_train[:3], 'out of ',len(y_train))

['tthk', 'ytrn', 'bhff']
x:
 [[116 116 104 107]
 [121 116 114 110]
 [ 98 104 102 102]] out of  3000

x_train:
 [[ 0.86440601  0.88369598 -0.75328185 -0.33426616]
 [ 1.54397048  0.88369598  0.58556852  0.07210902]
 [-1.58202609 -0.70030399 -1.02105192 -1.01155812]] out of  3000

y:
 [[19 19  7 10]
 [24 19 17 13]
 [ 1  7  5  5]
 ...
 [ 7  7  1 13]
 [ 3  0  9  3]
 [ 9  9  5 11]]

y_train:
 [[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
   0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
   0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
   0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
   0. 0. 0.]
  [0. 0. 0. 0

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [9]:
def get_sub_sub_set(y, c_ref, nb_chars):
    '''Retrieve the probalities of the i-th char'''
    nb_samples = len(y)
    
    yi = np.zeros((nb_samples, nb_letters), dtype=int)
    
    for s in range(nb_samples):
        for c in range(nb_chars):
            #print('ysl:',y[s][0][l_i])
            if c == c_ref:
                yi[s] = y[s, c]
                
    return yi

In [10]:
y_train2 = []
for c in range(nb_chars):
    yi_train = get_sub_sub_set(y_train, c, nb_chars)
    y_train2.append(yi_train)

In [11]:
y_train2[:3]

[array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 1, 0],
        [0, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]), array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]), array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [12]:
history = model.fit(x_train, y_train2, epochs=1000, batch_size=100,  verbose=1)

Epoch 1/1000
3000/3000 [==============================] - 0s 162us/step - loss: 11.6812 - output_0_loss: 2.9314 - output_1_loss: 2.9190 - output_2_loss: 2.9127 - output_3_loss: 2.9180 - output_0_acc: 0.1117 - output_1_acc: 0.0940 - output_2_acc: 0.1050 - output_3_acc: 0.0980
Epoch 2/1000
3000/3000 [==============================] - 0s 45us/step - loss: 10.2502 - output_0_loss: 2.5764 - output_1_loss: 2.5631 - output_2_loss: 2.5578 - output_3_loss: 2.5529 - output_0_acc: 0.1630 - output_1_acc: 0.1557 - output_2_acc: 0.1620 - output_3_acc: 0.1727
Epoch 3/1000
3000/3000 [==============================] - 0s 44us/step - loss: 9.5225 - output_0_loss: 2.3925 - output_1_loss: 2.3812 - output_2_loss: 2.3787 - output_3_loss: 2.3702 - output_0_acc: 0.2033 - output_1_acc: 0.1890 - output_2_acc: 0.1993 - output_3_acc: 0.1963
Epoch 4/1000
3000/3000 [==============================] - 0s 46us/step - loss: 9.0244 - output_0_loss: 2.2675 - output_1_loss: 2.2557 - output_2_loss: 2.2539 - output_3_loss: 

3000/3000 [==============================] - 0s 40us/step - loss: 5.3651 - output_0_loss: 1.3530 - output_1_loss: 1.3495 - output_2_loss: 1.3397 - output_3_loss: 1.3228 - output_0_acc: 0.4837 - output_1_acc: 0.4680 - output_2_acc: 0.4793 - output_3_acc: 0.4840
Epoch 31/1000
3000/3000 [==============================] - 0s 39us/step - loss: 5.3221 - output_0_loss: 1.3460 - output_1_loss: 1.3415 - output_2_loss: 1.3329 - output_3_loss: 1.3017 - output_0_acc: 0.4717 - output_1_acc: 0.4743 - output_2_acc: 0.4663 - output_3_acc: 0.4860
Epoch 32/1000
3000/3000 [==============================] - 0s 40us/step - loss: 5.2363 - output_0_loss: 1.3254 - output_1_loss: 1.3121 - output_2_loss: 1.3140 - output_3_loss: 1.2847 - output_0_acc: 0.4927 - output_1_acc: 0.4793 - output_2_acc: 0.4953 - output_3_acc: 0.5137
Epoch 33/1000
3000/3000 [==============================] - 0s 41us/step - loss: 5.1639 - output_0_loss: 1.3004 - output_1_loss: 1.2973 - output_2_loss: 1.3019 - output_3_loss: 1.2643 - outp

3000/3000 [==============================] - 0s 39us/step - loss: 3.8775 - output_0_loss: 0.9865 - output_1_loss: 0.9882 - output_2_loss: 0.9704 - output_3_loss: 0.9325 - output_0_acc: 0.6150 - output_1_acc: 0.6330 - output_2_acc: 0.6427 - output_3_acc: 0.6507
Epoch 60/1000
3000/3000 [==============================] - 0s 39us/step - loss: 3.8477 - output_0_loss: 0.9759 - output_1_loss: 0.9735 - output_2_loss: 0.9668 - output_3_loss: 0.9316 - output_0_acc: 0.6350 - output_1_acc: 0.6300 - output_2_acc: 0.6383 - output_3_acc: 0.6603
Epoch 61/1000
3000/3000 [==============================] - 0s 38us/step - loss: 3.7833 - output_0_loss: 0.9717 - output_1_loss: 0.9476 - output_2_loss: 0.9413 - output_3_loss: 0.9227 - output_0_acc: 0.6267 - output_1_acc: 0.6550 - output_2_acc: 0.6453 - output_3_acc: 0.6643
Epoch 62/1000
3000/3000 [==============================] - 0s 39us/step - loss: 3.7681 - output_0_loss: 0.9586 - output_1_loss: 0.9548 - output_2_loss: 0.9381 - output_3_loss: 0.9167 - outp

3000/3000 [==============================] - 0s 41us/step - loss: 2.9483 - output_0_loss: 0.7509 - output_1_loss: 0.7370 - output_2_loss: 0.7526 - output_3_loss: 0.7079 - output_0_acc: 0.7283 - output_1_acc: 0.7417 - output_2_acc: 0.7333 - output_3_acc: 0.7593
Epoch 89/1000
3000/3000 [==============================] - 0s 41us/step - loss: 2.9550 - output_0_loss: 0.7486 - output_1_loss: 0.7473 - output_2_loss: 0.7418 - output_3_loss: 0.7173 - output_0_acc: 0.7303 - output_1_acc: 0.7297 - output_2_acc: 0.7300 - output_3_acc: 0.7477
Epoch 90/1000
3000/3000 [==============================] - 0s 43us/step - loss: 2.9241 - output_0_loss: 0.7556 - output_1_loss: 0.7401 - output_2_loss: 0.7438 - output_3_loss: 0.6846 - output_0_acc: 0.7173 - output_1_acc: 0.7340 - output_2_acc: 0.7343 - output_3_acc: 0.7687
Epoch 91/1000
3000/3000 [==============================] - 0s 40us/step - loss: 2.8841 - output_0_loss: 0.7404 - output_1_loss: 0.7355 - output_2_loss: 0.7251 - output_3_loss: 0.6832 - outp

3000/3000 [==============================] - 0s 39us/step - loss: 2.3164 - output_0_loss: 0.6049 - output_1_loss: 0.5986 - output_2_loss: 0.5678 - output_3_loss: 0.5452 - output_0_acc: 0.8027 - output_1_acc: 0.7930 - output_2_acc: 0.8237 - output_3_acc: 0.8397
Epoch 118/1000
3000/3000 [==============================] - 0s 43us/step - loss: 2.3201 - output_0_loss: 0.6051 - output_1_loss: 0.5918 - output_2_loss: 0.5859 - output_3_loss: 0.5373 - output_0_acc: 0.7997 - output_1_acc: 0.8143 - output_2_acc: 0.8200 - output_3_acc: 0.8377
Epoch 119/1000
3000/3000 [==============================] - 0s 42us/step - loss: 2.2872 - output_0_loss: 0.5931 - output_1_loss: 0.5893 - output_2_loss: 0.5715 - output_3_loss: 0.5334 - output_0_acc: 0.8097 - output_1_acc: 0.8130 - output_2_acc: 0.8190 - output_3_acc: 0.8483
Epoch 120/1000
3000/3000 [==============================] - 0s 41us/step - loss: 2.2779 - output_0_loss: 0.5885 - output_1_loss: 0.5849 - output_2_loss: 0.5777 - output_3_loss: 0.5269 - o

...

3000/3000 [==============================] - 0s 42us/step - loss: 0.0035 - output_0_loss: 9.2114e-04 - output_1_loss: 9.1563e-04 - output_2_loss: 6.8600e-04 - output_3_loss: 9.3790e-04 - output_0_acc: 1.0000 - output_1_acc: 1.0000 - output_2_acc: 1.0000 - output_3_acc: 1.0000
Epoch 870/1000
3000/3000 [==============================] - 0s 44us/step - loss: 0.0032 - output_0_loss: 4.3364e-04 - output_1_loss: 7.8974e-04 - output_2_loss: 9.7115e-04 - output_3_loss: 0.0010 - output_0_acc: 1.0000 - output_1_acc: 1.0000 - output_2_acc: 1.0000 - output_3_acc: 1.0000
Epoch 871/1000
3000/3000 [==============================] - 0s 46us/step - loss: 0.0033 - output_0_loss: 0.0012 - output_1_loss: 8.7850e-04 - output_2_loss: 5.4267e-04 - output_3_loss: 6.0730e-04 - output_0_acc: 1.0000 - output_1_acc: 1.0000 - output_2_acc: 1.0000 - output_3_acc: 1.0000
Epoch 872/1000
3000/3000 [==============================] - 0s 44us/step - loss: 0.0027 - output_0_loss: 5.5386e-04 - output_1_loss: 7.8232e-04 - o

3000/3000 [==============================] - 0s 40us/step - loss: 0.0020 - output_0_loss: 3.9054e-04 - output_1_loss: 5.7959e-04 - output_2_loss: 5.5040e-04 - output_3_loss: 5.2240e-04 - output_0_acc: 1.0000 - output_1_acc: 1.0000 - output_2_acc: 1.0000 - output_3_acc: 1.0000
Epoch 898/1000
3000/3000 [==============================] - 0s 39us/step - loss: 0.0030 - output_0_loss: 8.5359e-04 - output_1_loss: 6.7576e-04 - output_2_loss: 3.8184e-04 - output_3_loss: 0.0011 - output_0_acc: 1.0000 - output_1_acc: 1.0000 - output_2_acc: 1.0000 - output_3_acc: 1.0000
Epoch 899/1000
3000/3000 [==============================] - 0s 40us/step - loss: 0.0020 - output_0_loss: 3.4433e-04 - output_1_loss: 5.8205e-04 - output_2_loss: 6.0388e-04 - output_3_loss: 5.0242e-04 - output_0_acc: 1.0000 - output_1_acc: 1.0000 - output_2_acc: 1.0000 - output_3_acc: 1.0000
Epoch 900/1000
3000/3000 [==============================] - 0s 40us/step - loss: 0.0024 - output_0_loss: 9.7191e-04 - output_1_loss: 4.3187e-04

3000/3000 [==============================] - 0s 43us/step - loss: 0.0016 - output_0_loss: 3.1869e-04 - output_1_loss: 3.7962e-04 - output_2_loss: 5.0391e-04 - output_3_loss: 4.0087e-04 - output_0_acc: 1.0000 - output_1_acc: 1.0000 - output_2_acc: 1.0000 - output_3_acc: 1.0000
Epoch 926/1000
3000/3000 [==============================] - 0s 42us/step - loss: 0.0023 - output_0_loss: 4.6964e-04 - output_1_loss: 9.5509e-04 - output_2_loss: 3.5103e-04 - output_3_loss: 5.3663e-04 - output_0_acc: 1.0000 - output_1_acc: 1.0000 - output_2_acc: 1.0000 - output_3_acc: 1.0000
Epoch 927/1000
3000/3000 [==============================] - 0s 42us/step - loss: 0.0018 - output_0_loss: 6.9354e-04 - output_1_loss: 3.9547e-04 - output_2_loss: 4.6965e-04 - output_3_loss: 2.1999e-04 - output_0_acc: 1.0000 - output_1_acc: 1.0000 - output_2_acc: 1.0000 - output_3_acc: 1.0000
Epoch 928/1000
3000/3000 [==============================] - 0s 44us/step - loss: 0.0019 - output_0_loss: 5.4506e-04 - output_1_loss: 3.0394

3000/3000 [==============================] - 0s 41us/step - loss: 0.0011 - output_0_loss: 2.1370e-04 - output_1_loss: 3.7161e-04 - output_2_loss: 2.5387e-04 - output_3_loss: 2.7394e-04 - output_0_acc: 1.0000 - output_1_acc: 1.0000 - output_2_acc: 1.0000 - output_3_acc: 1.0000
Epoch 954/1000
3000/3000 [==============================] - 0s 42us/step - loss: 0.0016 - output_0_loss: 5.0114e-04 - output_1_loss: 4.2281e-04 - output_2_loss: 5.2092e-04 - output_3_loss: 2.0276e-04 - output_0_acc: 1.0000 - output_1_acc: 1.0000 - output_2_acc: 1.0000 - output_3_acc: 1.0000
Epoch 955/1000
3000/3000 [==============================] - 0s 40us/step - loss: 0.0018 - output_0_loss: 3.3105e-04 - output_1_loss: 6.9187e-04 - output_2_loss: 2.9592e-04 - output_3_loss: 4.6627e-04 - output_0_acc: 1.0000 - output_1_acc: 1.0000 - output_2_acc: 1.0000 - output_3_acc: 1.0000
Epoch 956/1000
3000/3000 [==============================] - 0s 41us/step - loss: 0.0013 - output_0_loss: 6.1264e-04 - output_1_loss: 2.8775

3000/3000 [==============================] - 0s 40us/step - loss: 0.0012 - output_0_loss: 1.4972e-04 - output_1_loss: 4.2005e-04 - output_2_loss: 4.2907e-04 - output_3_loss: 1.6572e-04 - output_0_acc: 1.0000 - output_1_acc: 1.0000 - output_2_acc: 1.0000 - output_3_acc: 1.0000
Epoch 982/1000
3000/3000 [==============================] - 0s 43us/step - loss: 0.0011 - output_0_loss: 4.8185e-04 - output_1_loss: 1.4330e-04 - output_2_loss: 1.6315e-04 - output_3_loss: 2.6707e-04 - output_0_acc: 1.0000 - output_1_acc: 1.0000 - output_2_acc: 1.0000 - output_3_acc: 1.0000
Epoch 983/1000
3000/3000 [==============================] - 0s 43us/step - loss: 0.0016 - output_0_loss: 3.0861e-04 - output_1_loss: 5.9614e-04 - output_2_loss: 4.4337e-04 - output_3_loss: 2.4998e-04 - output_0_acc: 1.0000 - output_1_acc: 1.0000 - output_2_acc: 1.0000 - output_3_acc: 1.0000
Epoch 984/1000
3000/3000 [==============================] - 0s 42us/step - loss: 9.5990e-04 - output_0_loss: 2.7595e-04 - output_1_loss: 1.

In [13]:
def print_readable_prediction(prediction, nb_words):
    
    words = [''] * nb_words
    
    c_i = 0
    for char_prediction in prediction:

        s_i = 0
        for sample_char_prediction in char_prediction:

            i_letter = 0
            best_value = -float('inf')
            best_letter = -1
            for letter_prediction in sample_char_prediction:
                if letter_prediction > best_value:
                    best_value = letter_prediction
                    best_letter = i_letter
                i_letter += 1
            words[s_i] += str(best_letter)
            words[s_i] += '-'
            s_i += 1
        c_i += 1
    print(words)

In [14]:
nb_words = 2

x_test = create_inputs(nb_words, nb_chars, nb_letters)
display_x(x_test)
print("x_test=\n", x_test)

x_test_n  = scaler.transform(x_test)
print("x_test_mean=\n", x_test_n)
print('-->')

prediction = model.predict(x_test_n)
#print(prediction)
print('prediction')
print_readable_prediction(prediction, nb_words)

print('check prediction')
y_test = encrypt(x_test, nb_words, nb_chars)
print("y_test=\n", y_test)

['ilul', 'xqvn']
x_test=
 [[105 108 117 108]
 [120 113 118 110]]
x_test_mean=
 [[-0.63063583 -0.172304    0.98722364 -0.19880777]
 [ 1.40805759  0.48769599  1.12110867  0.07210902]]
-->
prediction
['8-11-20-11-', '23-16-21-13-']
check prediction
y_test=
 [[ 8 11 20 11]
 [23 16 21 13]]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
